<a href="https://colab.research.google.com/github/rishavsen1/gtfs_filtering/blob/main/trips_vid_time_stop_block_GTFS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
!unzip bustime_chattanooga_jan_2022.json.zip
!unzip 2021-10-24n.zip

Archive:  bustime_chattanooga_jan_2022.json.zip
replace bustime_chattanooga_jan_2022.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
Archive:  2021-10-24.zip
  inflating: agency.txt              
  inflating: calendar.txt            
  inflating: calendar_dates.txt      
  inflating: routes.txt              
  inflating: shapes.txt              
  inflating: stop_times.txt          
  inflating: stops.txt               
  inflating: trips.txt               


In [17]:
import json
with open("bustime_chattanooga_jan_2022.json", "r") as f:
    data = json.load(f)

dated = []
for record in data:
    tmstmp = record["tmstmp"]
    if "20220111" in tmstmp:

        dated.append(record)



In [18]:
dated

[{'_id': {'$oid': '61dd0ee34a63d75fa69c05fe'},
  'des': 'DOWNTOWN',
  'dly': 'false',
  'geometry': {'coordinates': ['-85.30767059326172', '34.992942810058594'],
   'type': 'Point'},
  'hdg': '24',
  'lat': '34.992942810058594',
  'lon': '-85.30767059326172',
  'mode': '0',
  'oid': '161067',
  'or': 'false',
  'origtatripno': '204959',
  'pdist': '8025',
  'pid': '919',
  'psgld': 'EMPTY',
  'rid': '46',
  'rt': '1',
  'spd': '20',
  'srvtmstmp': '20220111 00:00',
  'tablockid': '1561',
  'tatripid': '204959',
  'timestamp': 1641877200,
  'tmstmp': '20220111 00:00',
  'tripdyn': '0',
  'vid': '112',
  'zone': None},
 {'_id': {'$oid': '61dd0f014a63d75fa69c0601'},
  'des': None,
  'dly': 'false',
  'geometry': {'coordinates': ['-85.269788', '35.05660966666667'],
   'type': 'Point'},
  'hdg': '119',
  'lat': '35.05660966666667',
  'lon': '-85.269788',
  'mode': '0',
  'oid': 'N/A',
  'or': 'true',
  'origtatripno': None,
  'pdist': '0',
  'pid': '-1',
  'psgld': 'EMPTY',
  'rid': 'N/A',


In [21]:
data = dated
data2 = []
for record in data:
    trp = record["origtatripno"]
    if trp:
        data2.append(trp)

unique_trp = list(set(data2))
unique_trp.sort()


In [25]:
for val in data:
  val['coords'] = val['geometry']['coordinates']

In [27]:
from datetime import time
data_gen = []
vid = []
prev_trip_id = None
for record in data:
    trip_id = record["origtatripno"]
    if prev_trip_id != trip_id:
        vid = record['vid']
        srvtmstmp = record['srvtmstmp']
        data_gen.append({"trip_id": trip_id, "vid":vid, "start_time":srvtmstmp})
        prev_trip_id = trip_id

# unique_trp = list(set(data_gen))
# unique_trp.sort()
# data_gen
# print(len(unique_trp))

## Clever vid, trip id and GTFS times

In [29]:
import pandas as pd
data_gen_df = pd.DataFrame(data_gen, columns=["trip_id", 'vid', 'start_time'])
# data_gen_df

In [30]:
df_mod = data_gen_df.drop_duplicates(subset=["trip_id", "vid"]).reset_index(inplace=False).dropna()
df_mod["trip_id"] = df_mod["trip_id"]+"020"
df_mod = df_mod.drop(['index'], axis=1)
# df_mod = df_mod[["trip_id", "vid", 'start_time']]
df_mod = df_mod[df_mod.start_time > '20220111 04:00']

In [1]:
!pip install geopandas
!pip install gtfs_functions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 5.1 MB/s 
     |████████████████████████████████| 16.7 MB 36.9 MB/s 
     |████████████████████████████████| 6.3 MB 48.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 187 kB 5.3 MB/s 


In [31]:
import gtfs_functions as gtfs
routes, stops, stop_times, trips, shapes = gtfs.import_gtfs('./2021-10-24.zip')

In [32]:
import time
def val_to_time(val):
  return (time.strftime('%H:%M:%S', time.gmtime(val)))

In [47]:
df1 = df_mod
for unique in unique_trp:
    temp = stop_times[stop_times.trip_id == unique+'020'].sort_values('arrival_time')
    start = temp.arrival_time.iloc[0]
    end = temp.arrival_time.iloc[-1]
    t2 = df_mod.loc[df_mod.trip_id == unique+'020']
    df1.loc[t2.index.tolist()[0], 'gtfs_start_time'] = val_to_time(start)
    df1.loc[t2.index.tolist()[0], 'gtfs_end_time'] = val_to_time(end)
    df1.loc[t2.index.tolist()[0], 'first_stop_id'] = temp.stop_id.iloc[0]
    df1.loc[t2.index.tolist()[0], 'last_stop_id'] = temp.stop_id.iloc[-1]

df1 = df1.reset_index(drop=True)

In [53]:
trips_txt = pd.read_csv('./trips.txt')

In [60]:
trips_txt.trip_id = trips_txt.trip_id.astype(str)

In [62]:
df1.merge(trips_txt.groupby(['block_id', 'trip_id']).first().reset_index()[['trip_id', 'block_id', 'block_name']], on='trip_id', how='left')

,trip_id,vid,start_time,gtfs_start_time,gtfs_end_time,first_stop_id,lasst_stop_id,last_stop_id,block_id,block_name
0,206311020,504,20220111 04:21,04:21:00,05:30:00,354,1876,1876,4902,402
1,206129020,113,20220111 04:39,04:41:00,05:25:00,2092,1925,1925,3602,301
2,205253020,150,20220111 04:45,04:51:00,05:30:00,354,1710,1710,1202,1301
3,204922020,123,20220111 04:45,04:51:00,05:30:00,354,95,95,502,102
4,206312020,501,20220111 04:46,04:51:00,06:00:00,354,1876,1876,5002,403
...,...,...,...,...,...,...,...,...,...,...
708,206345020,111,20220111 23:25,23:30:00,00:40:00,1878,690,690,6302,458
709,205173020,154,20220111 23:25,23:30:00,23:55:00,145,288,288,2802,155
710,204923020,126,20220111 23:37,23:30:00,23:55:00,1351,95,95,3002,156
711,204959020,126,20220111 23:57,23:55:00,00:40:00,95,690,690,3002,156


In [64]:
df1.to_csv('trips_vid_time_stops_GTFS_20220111.csv', index=False)